In [1]:
import csv
import itertools
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

In [2]:
%matplotlib inline
matplotlib.rcParams['figure.max_open_warning'] = 100
matplotlib.rcParams['lines.linewidth'] = 1.0

# Grupos

## Socios activos 2016

In [3]:
# 1.       CTA: categoría para niños y jóvenes hasta bachillerato
# 2.       CTI: categoría infantil, hasta doce años y solo es para Bogotá (blaa –biblioteca Luis Ángel Arango)
# 3.       CTB: público general (solo permite préstamo de libros)
# 4.       CTC: público en general (permite préstamo de libros y material audiovisual)
# 5.       CTF: categoría familiar, pueden acceder a los beneficios hasta tres personas del grupo familiar que se asocia
# 6.       CE1, CE2, CE2: categorías empresariales (solo para instituciones)
# 7.       CIS: categoría Inclusión Social
# 8.       CTU: categoría Universitaria
# 9.       VIP: Investigadores, préstamo de 30 materiales (libros y material audiovisual)

#      Categoría de lector: CE1 Categoría empresarial                                   218
#      Categoría de lector: CE2 Categoría empresarial 2                                   5
#      Categoría de lector: CEM Categoría Empleado/Pensionado                          1593
#      Categoría de lector: CIN Categoría Interbibliotecario                            128
#      Categoría de lector: CIP Categoría Investigador                                  351
#      Categoría de lector: CIS Categoría Inclusión Preferencial                      15974
#      Categoría de lector: CIV Categoría Investigadores                                 72
#      Categoría de lector: CML Categoría Mediador de Proyectos Culturales                2
#      Categoría de lector: CSU Categoría Sucursal                                       91
#      Categoría de lector: CTA Categoría Básica A                                    10610
#      Categoría de lector: CTB Categoría B                                            1359
#      Categoría de lector: CTC Categoría C                                             224
#      Categoría de lector: CTF Categoría familiar                                     1105
#      Categoría de lector: CTI Categoría Infantil                                     1004
#      Categoría de lector: CTU Categoría General                                     17402
#      Categoría de lector: PTE PROCESOS TÉCNICOS                                        95

In [4]:
%%time

dtypes = {
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Edad': 'category',
}

perfiles_df = pd.read_csv(
    'valores_grupos_2016.csv', header=0, dtype=dtypes)
# perfiles_df = perfiles_df.fillna('')

CPU times: user 64.8 ms, sys: 8.21 ms, total: 73 ms
Wall time: 71.9 ms


In [5]:
perfiles_df.shape

(50272, 9)

In [6]:
perfiles_df.head()

,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Edad
0,BLAA,OTRO,chapinero,bogota,CIN,universidad,OTRO,OTRO,OTRO
1,BLAA,sra.,OTRO,bogota,CEM,pensionado,OTRO,OTRO,51-64
2,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,OTRO
3,ARM,sr.,OTRO,OTRO,CEM,empleado,profesional,economía,51-64
4,BLAA,sr.,soledad,bogota,CEM,OTRO,OTRO,OTRO,OTRO


# Bogotá

In [16]:
perfiles_bogota_df = perfiles_df[perfiles_df['Ciudad'] == 'bogota']

In [17]:
perfiles_bogota_df.shape

(23496, 9)

In [18]:
perfiles_bogota_df.head()

,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Edad
0,BLAA,OTRO,chapinero,bogota,CIN,universidad,OTRO,OTRO,OTRO
1,BLAA,sra.,OTRO,bogota,CEM,pensionado,OTRO,OTRO,51-64
2,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,OTRO
4,BLAA,sr.,soledad,bogota,CEM,OTRO,OTRO,OTRO,OTRO
5,BLAA,OTRO,galerias,bogota,CIN,OTRO,OTRO,OTRO,OTRO


## Campos a estudiar

### Porcentajes sin valores

In [25]:
columns = sorted(dtypes)

In [26]:
for c in columns:
    cns = perfiles_bogota_df[c].value_counts()
    total = cns.sum()
    otros = 0
    if 'OTRO' in cns:
        otros = cns.ix['OTRO']
    perc = '{:.2%}'.format(otros/total)
    unique = perfiles_bogota_df[c].nunique()
    print(c, perc, unique)

Barrio 8.59% 1601
CATEG 0.00% 14
Ciudad 0.00% 1
Edad 15.90% 7
Nivel Estudio1 15.07% 13
Ocupación1 2.92% 17
Sucursal 0.00% 28
Tratamiento 8.08% 3
Área de Estudio1 26.08% 113


##### => Estudiar nivel de estudios (Nivel Estudio1, 15.07% sin valor) y área de estudios (Área de Estudio1, 26.33% sin valor) en Bogotá por barrios

# Otras ciudades

In [27]:
perfiles_otras_df = perfiles_df[perfiles_df['Ciudad'] != 'bogota']

In [28]:
perfiles_otras_df.shape

(26776, 9)

In [29]:
perfiles_otras_df.head()

,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Edad
3,ARM,sr.,OTRO,OTRO,CEM,empleado,profesional,economía,51-64
18,BLAA,sr.,juriscol,OTRO,CEM,OTRO,OTRO,OTRO,OTRO
20,IBA,sra.,claret,ibague,CIS,estudiante,técnico,OTRO,31-40
27,OTRO,OTRO,pensionado,OTRO,CEM,OTRO,OTRO,OTRO,OTRO
28,PER,sra.,OTRO,pereira,CEM,OTRO,OTRO,OTRO,51-64


## Campos a estudiar

### Porcentajes sin valores

In [30]:
columns = sorted(dtypes)

In [31]:
for c in columns:
    cns = perfiles_otras_df[c].value_counts()
    total = cns.sum()
    otros = 0
    if 'OTRO' in cns:
        otros = cns.ix['OTRO']
    perc = '{:.2%}'.format(otros/total)
    unique = perfiles_otras_df[c].nunique()
    print(c, perc, unique)

Barrio 15.19% 2535
CATEG 0.00% 15
Ciudad 10.52% 82
Edad 10.79% 7
Nivel Estudio1 10.59% 13
Ocupación1 4.28% 17
Sucursal 0.04% 31
Tratamiento 13.17% 3
Área de Estudio1 64.77% 113


##### => Estudiar nivel de estudios (Nivel Estudio1, 10.07% sin valor) pero no área de estudios (Área de Estudio1, 64.77% sin valor) en otras ciudades por barrios